In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import random
import shutil


In [13]:
dataset_dir = '/content/drive/MyDrive/Metal Detect/dataset/Copper and Brass/'
train_dir = '/content/drive/MyDrive/Metal Detect/dataset/Copper and Brass/train'
val_dir = '/content/drive/MyDrive/Metal Detect/dataset/Copper and Brass/valid'
test_dir = '/content/drive/MyDrive/Metal Detect/dataset/Copper and Brass/test'


In [14]:
# Create train, validation, and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [16]:
images = os.listdir(os.path.join(dataset_dir, 'images'))
random.shuffle(images)


train_size = int(0.7 * len(images))
val_size = int(0.2 * len(images))

train_images = images[:train_size]
val_images = images[train_size:train_size + val_size]
test_images = images[train_size + val_size:]

In [17]:
for img in train_images:
    shutil.move(os.path.join(dataset_dir, 'images', img), os.path.join(train_dir, img))

for img in val_images:
    shutil.move(os.path.join(dataset_dir, 'images', img), os.path.join(val_dir, img))

for img in test_images:
    shutil.move(os.path.join(dataset_dir, 'images', img), os.path.join(test_dir, img))

In [22]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
input_shape = (224, 224, 3)
num_classes = 4  # Copper, Brass, Still, Non-metal

In [24]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

In [26]:
train_generator = ImageDataGenerator().flow_from_directory(
    '/content/drive/MyDrive/Metal Detect/dataset/Copper and Brass/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = ImageDataGenerator().flow_from_directory(
    '/content/drive/MyDrive/Metal Detect/dataset/Copper and Brass/valid',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=10)

ValueError: Asked to retrieve element 0, but the Sequence has length 0